In [2]:
import pandas as pd
import statsmodels.api as sm
from pathlib import Path

BASE_DIR = Path.cwd().parent.parent.parent

DATA_RAW = BASE_DIR / "data" / "raw"
DATA_PROCESSED = BASE_DIR / "data" / "processed"

prices = pd.read_csv(DATA_PROCESSED / "prices_with_returns.csv")
prices["date"] = pd.to_datetime(prices["date"])
prices["ticker"] = prices["ticker"].astype(str).str.upper()

prices.head()


,date,ticker,adj_close,return,market_return
0,2010-01-04,ATVI,17.663647,NaN,NaN
1,2010-01-05,ATVI,18.159983,0.028099,0.003116
2,2010-01-06,ATVI,17.692842,-0.025724,0.000546
3,2010-01-07,ATVI,17.556591,-0.007701,0.004001
4,2010-01-08,ATVI,17.906948,0.019956,0.002882


In [3]:
def estimate_alpha_beta(prices, ticker, market_ticker="SP500"):
    df = prices[prices["ticker"] == ticker].dropna(subset=["return", "market_return"])
    
    if df.empty:
        return None, None
    
    X = sm.add_constant(df["market_return"])
    y = df["return"]
    
    model = sm.OLS(y, X).fit()
    alpha = model.params["const"]
    beta = model.params["market_return"]
    
    return alpha, beta


In [4]:
estimate_alpha_beta(prices, "TTWO")


(np.float64(0.0005460214326912661), np.float64(0.9393538682534797))

In [5]:
tickers = prices["ticker"].unique()
ab_table = {}

for t in tickers:
    alpha, beta = estimate_alpha_beta(prices, t)
    ab_table[t] = {"alpha": alpha, "beta": beta}

ab_table


{'ATVI': {'alpha': np.float64(0.00039012809726157087),
  'beta': np.float64(0.8264003793929274)},
 'EA': {'alpha': np.float64(0.00039012809726157087),
  'beta': np.float64(0.8264003793929274)},
 'NTDOY': {'alpha': np.float64(-6.137861502535158e-05),
  'beta': np.float64(0.6442097175724608)},
 'SP500': {'alpha': np.float64(1.2197274440461925e-19),
  'beta': np.float64(0.9999999999999989)},
 'TTWO': {'alpha': np.float64(0.0005460214326912661),
  'beta': np.float64(0.9393538682534797)},
 'UBSFY': {'alpha': np.float64(0.00026630182154660737),
  'beta': np.float64(0.636621344180584)}}

In [6]:
events = pd.read_csv(DATA_PROCESSED / "events_with_returns.csv")
events["event_date"] = pd.to_datetime(events["event_date"])
events["trading_date"] = pd.to_datetime(events["trading_date"])
events["ticker"] = events["ticker"].astype(str).str.upper()

events.head()

KeyError: 'event_date'

In [ ]:
def compute_ar(row, ab_table):
    ticker = row["ticker_x"]
    alpha = ab_table[ticker]["alpha"]
    beta = ab_table[ticker]["beta"]
    
    er = alpha + beta * row["market_return"]   # expected return
    ar = row["return"] - er                   # abnormal return
    return ar

events["AR_event"] = events.apply(lambda r: compute_ar(r, ab_table), axis=1)
events[["event_id", "ticker_x", "event_date", "trading_date", "AR_event"]].head()

,event_id,ticker_x,event_date,trading_date,AR_event
0,ATVI_2019_CODMOBILE_LAUNCH,ATVI,2019-10-01,2019-10-01,-0.001198
1,ATVI_2019_CODMW_RELEASE,ATVI,2019-10-25,2019-10-25,-0.000318
2,ATVI_2020_WARCRAFT3_REFORGED,ATVI,2020-01-28,2020-01-28,0.003422
3,ATVI_2020_WARZONE_LAUNCH,ATVI,2020-03-10,2020-03-10,-0.016937
4,ATVI_2021_DIABLO2_RESURRECTED,ATVI,2021-09-23,2021-09-23,0.001370


In [ ]:
prices["expected_return"] = prices.apply(
    lambda r: ab_table[r["ticker"]]["alpha"] + ab_table[r["ticker"]]["beta"] * r["market_return"]
    if pd.notnull(r["market_return"]) else None,
    axis=1
)

prices["AR"] = prices["return"] - prices["expected_return"]


In [ ]:
def compute_car(prices, ticker, trading_date, window=(-1, 1)):
    start = trading_date + pd.Timedelta(days=window[0])
    end = trading_date + pd.Timedelta(days=window[1])
    
    df = prices[(prices["ticker"] == ticker) &
                (prices["date"] >= start) &
                (prices["date"] <= end)]
    return df["AR"].sum()


In [ ]:
events["CAR_m1_p1"] = events.apply(
    lambda r: compute_car(prices, r["ticker_x"], r["trading_date"], window=(-1, 1)),
    axis=1
)

events[["event_id", "ticker_x", "event_date", "trading_date", "AR_event", "CAR_m1_p1"]].head()


,event_id,ticker_x,event_date,trading_date,AR_event,CAR_m1_p1
0,ATVI_2019_CODMOBILE_LAUNCH,ATVI,2019-10-01,2019-10-01,-0.001198,0.005278
1,ATVI_2019_CODMW_RELEASE,ATVI,2019-10-25,2019-10-25,-0.000318,0.000526
2,ATVI_2020_WARCRAFT3_REFORGED,ATVI,2020-01-28,2020-01-28,0.003422,0.003371
3,ATVI_2020_WARZONE_LAUNCH,ATVI,2020-03-10,2020-03-10,-0.016937,0.001569
4,ATVI_2021_DIABLO2_RESURRECTED,ATVI,2021-09-23,2021-09-23,0.001370,0.004392


In [ ]:
# Save the final dataset
out_path = DATA_PROCESSED / "events_with_car.csv"
events.to_csv(out_path, index=False)
out_path


PosixPath('/files/capstone_project/game-market-event-analyzer/data/processed/events_with_car.csv')